In [2]:
import cv2
import os
import matplotlib.pyplot as plt
import glob
import face_recognition
import numpy as np
import joblib
import uuid
import seaborn as sns

In [3]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model, Model
from keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten
import imutils

In [33]:
SIZE = 224

In [34]:
DetectorLabels = ['Mask','No Mask']

In [11]:
train_labels = []
for directory_path in glob.glob(r"D:\ML DEEP LEARNING FACE\Masked Face Recognition\database\Train\*"):
    label = directory_path.split("\\")[-1]
    train_labels.append(label)
train_labels = np.array(train_labels)

In [9]:
# Training data and labels into respective lists
train_images = []
train_labels = []

for directory_path in glob.glob("database/Train/*"):
    label = directory_path.split("\\")[-1]
    # print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        # print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
        
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [12]:
len(train_labels)

3

In [13]:
#Encode labels from text to integers.
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
encoder.fit(train_labels)
train_labels_encoded = encoder.transform(train_labels)

In [36]:
x_train, y_train = train_images, train_labels_encoded
# Normalize pixel values to between 0 and 1
x_train = x_train / 255.0

In [37]:
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)

In [38]:
# Loading model
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

In [39]:
for layer in VGG_model.layers:
	layer.trainable = False
    
x=Flatten()(VGG_model.output)
VGG_model = Model(inputs=VGG_model.input, outputs=x)

VGG_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 240, 240, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 240, 240, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 240, 240, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 120, 120, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 120, 120, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 120, 120, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 60, 60, 128)       0     

In [40]:
# feature_extractor=VGG_model.predict(x_train)
features=VGG_model.predict(x_train)

2/2 [==============================] - 14s 6s/step


In [41]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 100, random_state = 42)

In [42]:
RF_model.fit(features, y_train)

RandomForestClassifier(n_estimators=120, random_state=42)

In [6]:
def detect_and_predict_mask(frame, faceNet, maskNet):
	# grab the dimensions of the frame and then construct a blob
	# from it
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
		(104.0, 177.0, 123.0))

	# pass the blob through the network and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()
	print(detections.shape)

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the detection
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)

			# add the face and bounding boxes to their respective
			# lists
			faces.append(face)
			locs.append((startX, startY, endX, endY))

	# only make a predictions if at least one face was detected
	if len(faces) > 0:
		# for faster inference we'll make batch predictions on *all*
		# faces at the same time rather than one-by-one predictions
		# in the above `for` loop
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

	# return a 2-tuple of the face locations and their corresponding
	# locations
	return (locs, preds)

In [7]:
protoPath = "D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\face_detector\\deploy.prototxt"
weightsPath = "D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\face_detector\\res10_300x300_ssd_iter_140000.caffemodel"

faceNet = cv2.dnn.readNet(protoPath, weightsPath)

In [8]:
maskNet = load_model('D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\mask_detector.model')

In [45]:
VGG_model.save('featuresVGG.h5')
joblib.dump(RF_model, "RFModel.joblib")

['RFModel.joblib']

In [46]:
VGG_model = load_model("D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\featuresVGG.h5")
RF_model = joblib.load("D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\RFModel.joblib")

In [14]:
model = load_model('model_vgg162.h5')

In [59]:
def predict(img):
    if img is None:
        return 'Unknown'
    img = cv2.resize(img,(SIZE,SIZE))
    input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
    prediction_RF=model.predict(input_img)
    prediction_RF = np.argmax(prediction_RF, axis=1)
    prediction_RF = encoder.inverse_transform([prediction_RF]) if prediction_RF<3 else None
    return str(prediction_RF)

In [47]:
# def predict(img):
#     if img is None:
#         return 'Unknown'
#     img = cv2.resize(img,(SIZE,SIZE))
#     input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
#     input_img_feature=VGG_model.predict(input_img)
#     prediction_RF = RF_model.predict(input_img_feature)[0]
#     # prediction_RF = RF_model.predict_proba(input_img_feature)
#     prediction_RF = encoder.inverse_transform([prediction_RF])  #Reverse the label encoder to original name
#     return str(prediction_RF)
#     # return prediction_RF

In [17]:
img = cv2.imread(r'D:\ML DEEP LEARNING FACE\Masked Face Recognition\database\Test\Geetansh\IMG_20220903_224600.jpg')
print(predict(img))

1/1 [==============================] - 0s 373ms/step
['Geetansh']


In [20]:
cap = cv2.VideoCapture(0)

while True:
	_ , frame = cap.read()

	# detect faces with mask or not
	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

	# loop over the detected face locations
	for (box, pred) in zip(locs, preds):
		(startX, startY, endX, endY) = box
		(mask, withoutMask) = pred

		label = "Mask" if mask > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		cv2.putText(frame, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY-120), (0,0,0), 2)
		img = frame[startY:endY-120,startX:endX]
		
		# cv2.imshow("cut",img)
		name = predict(img)
		if img is not None:
			cv2.putText(frame, name, (endX-20, endY+10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
  
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF

	if key == ord("q"):
		break

cap.release()
cv2.destroyAllWindows()

(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
1/1 [==============================] - 0s 274ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 383ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 410ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 305ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 334ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 314ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 326ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 262ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 304ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 426ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 334ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 264ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 391ms/step
(1, 1, 200,

In [19]:
featureDetector = load_model(r'face_detector\featuresVGGDetector.h5')
RFDetector = joblib.load(r'face_detector\RFModelDetector.joblib')

In [48]:
def detect(img):
    if img is None:
        return None
    img = cv2.resize(img,(224,224))
    input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
    input_img_feature=featureDetector.predict(input_img)
    prediction_RF = RFDetector.predict(input_img_feature)[0]
    prediction = RFDetector.predict_proba(input_img_feature)
    return DetectorLabels[prediction_RF],prediction

In [54]:
img = cv2.imread(r'D:\ML DEEP LEARNING FACE\Masked Face Recognition\data\MaskImages\without_mask\0_0_caizhuoyan_0014.jpg')
# max = np.max(predict(img))
label, predictions = detect(img)
print(label)
print(np.max(predictions))

1/1 [==============================] - 0s 292ms/step
No Mask
0.78


In [66]:
cap = cv2.VideoCapture(0)
cascade_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while cap.isOpened():
    ret,frame=cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    detection = cascade_classifier.detectMultiScale(gray,1.3,5)
    
    for (x,y,w,h) in detection:
        # (x,y,w,h)=detection[0]        
        frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        img_crop=frame[y:y+h,x:x+w]
        label, predictions = detect(img_crop)
        img = frame[y:y+h-120,x:x+w]
        name = predict(img)
        
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, np.max(predictions) * 100)

        cv2.putText(frame, label, (x, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h-120), (0,0,0), 2)
        cv2.putText(frame, name, (x+w-20, y+h+10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        
        
    cv2.imshow('frame',frame)
    
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 323ms/step


c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1/1 [==============================] - 0s 252ms/step


c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1/1 [==============================] - 0s 261ms/step


...

c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1/1 [==============================] - 0s 277ms/step


c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1/1 [==============================] - 0s 300ms/step


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
